<a href="https://colab.research.google.com/github/TienLungSun/Intelligent-Robot/blob/main/LearnPPO-AC/3.%20NN%20with%20policy%20interacts%20with%203D%20Ball%20to%20collect%20training%20data%20(MLAgent_10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import numpy as np

from mlagents_envs.environment import UnityEnvironment

In [ ]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print(device, torch.cuda.get_device_name(0))
else:
    device= torch.device("cpu")
    print(device)

cuda GeForce GTX 1660 SUPER


In [ ]:
# state = input of NN, action = output of NN
N_STATES  = 8
N_ACTIONS =2

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)

In [ ]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        
        self.actor = nn.Sequential(
            nn.Linear(N_STATES, 128),
            nn.LayerNorm(128),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.Linear(128, N_ACTIONS)
        )
        self.log_std = nn.Parameter(torch.ones(1, N_ACTIONS) * 0.0)
        self.apply(init_weights)
    
    def forward(self, x):
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist

In [ ]:
net = Net().to(device)

### Practice collecting data from agents 

In [ ]:
env = UnityEnvironment(file_name= None, base_port=5004)

In [ ]:
env.reset()
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]
print(behaviorName)

3DBall?team=0


In [ ]:
NoAgents = 3 #My Unity scene has three training environment
NoSteps = 20

In [ ]:
#practice to create a [NoAgents, 1] tensor
a = torch.FloatTensor([[0]]*NoAgents) 
print(a, a.shape)

tensor([[0.],
        [0.],
        [0.]]) torch.Size([3, 1])


In [ ]:
lst = [a]*NoSteps
print(len(lst), lst[0].shape)

20 torch.Size([3, 1])


In [ ]:
#practice to create a [NoAgents, NoActions] tensor
b = torch.FloatTensor([[0]*N_ACTIONS]*NoAgents) 
print(b, b.shape)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]]) torch.Size([3, 2])


In [ ]:
c = torch.FloatTensor([[0]*N_STATES]*NoAgents) 
print(c, c.shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]) torch.Size([3, 8])


In [ ]:
#define list to record interaction information
values = rewards = masks = [a]*NoSteps
log_probs = actions = [b]*NoSteps
states =[c]*NoSteps

print(len(log_probs), log_probs[0].shape)
print(len(values), values[0].shape)
print(len(rewards), rewards[0].shape)
print(len(masks), masks[0].shape)
print(len(states), states[0].shape)
print(len(actions), actions[0].shape)

20 torch.Size([3, 2])
20 torch.Size([3, 1])
20 torch.Size([3, 1])
20 torch.Size([3, 1])
20 torch.Size([3, 8])
20 torch.Size([3, 2])


In [ ]:
#find current agent state in decision and terminal steps
DecisionSteps, TerminalSteps = env.get_steps(behaviorName)

In [ ]:
DecisionStepsAgetnIds = list(DecisionSteps.agent_id)
print(DecisionStepsAgetnIds)

[0, 1, 2]


In [ ]:
#check whether current decision and terminal steps contain all agents
DecisionStepsAgetnIds = list(DecisionSteps.agent_id)
TerminalStepsAgentIds = list(TerminalSteps.agent_id)
lst = DecisionStepsAgetnIds + TerminalStepsAgentIds
unique_AgentIDs = list(set(lst)) # set = distinct values in a list
print(unique_AgentIDs)

[0, 1, 2]


In [ ]:
if(len(unique_AgentIDs) == NoAgents):
    print("This step includes all agents")
else:
    print("This step misses some agents")

This step includes all agents


In [ ]:
#reward
r = DecisionSteps.reward 
print(r)

[0. 0. 0.]


In [ ]:
torch.FloatTensor(r).unsqueeze(1)

tensor([[0.],
        [0.],
        [0.]])

In [ ]:
#practice data collection
s = DecisionSteps.obs[0]  
s = torch.FloatTensor(s).to(device)    
dist = net(s)
a = dist.sample() 
log_prob = dist.log_prob(a)
r = DecisionSteps.reward 
r = torch.FloatTensor(r).unsqueeze(1)
step = 0

for idx in range(len(DecisionSteps)):
    AgentID = DecisionSteps.agent_id[idx] 
    states[step][AgentID]=s[idx]
    values[step][AgentID]= 0.2 # value will be estimated by critic NN later 
    actions[step][AgentID]=a[idx]
    log_probs[step][AgentID]=log_prob[idx]
    rewards[step][AgentID]= r[idx]
    masks[step][AgentID]= 1

In [ ]:
env.close()

### Now run a loop to collect 20 steps of (s, a, r, s1) data from agents

In [ ]:
env = UnityEnvironment(file_name= None, base_port=5004)

In [ ]:
env.reset()
behaviorNames = list(env.behavior_specs.keys())
behaviorName = behaviorNames[0]

In [ ]:
# collect (s,a,r,s1) from 20 good interactions with Unity
a = torch.FloatTensor([[0]]*NoAgents) #create NoAgents by 1 tensor
b = torch.FloatTensor([[0]*N_ACTIONS]*NoAgents) 
c = torch.FloatTensor([[0]*N_STATES]*NoAgents) 
values = rewards = masks = [a]*NoSteps
log_probs = actions = [b]*NoSteps
states =[c]*NoSteps

step = 0  #index 0
DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
while(step < NoSteps): #try to run 20 good steps
    if(len(DecisionSteps) == 0): #we have no decision agents
        print("step ", step, "has no decision agents !")
        print("Decisin agents", list(DecisionSteps.agent_id))
        print("Terminal agents", list(TerminalSteps.agent_id))
        print("Reset the training environment!")
        env.reset() 
        DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
        #continue next while loop without increase step
    else:  # we have decision agents
        lst = list(DecisionSteps.agent_id) + list(TerminalSteps.agent_id)
        if(len(list(set(lst))) != NoAgents): 
            #if this step misses some agents, then only interacts with Unity 
            #but do not collect data
            s = DecisionSteps.obs[0]  
            s = torch.FloatTensor(s).to(device)    
            dist = net(s)
            a = dist.sample() 
            env.set_actions(behaviorName, a.cpu().detach().numpy() )   
            env.step()
            DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
            #continue next while loop without increase step
        else: 
            #this step includes all agents, collect (s, a, r, s1)
            s = DecisionSteps.obs[0]  
            s = torch.FloatTensor(s).to(device)         
            dist = net(s)
            a = dist.sample()
            log_prob = dist.log_prob(a)
            for idx in range(len(DecisionSteps)):
                #find decision agents and record their state, value and actions
                AgentID = DecisionSteps.agent_id[idx]
                states[step][AgentID]=s[idx]
                values[step][AgentID]= 0.2 # value will be estimated by critic NN later 
                actions[step][AgentID]=a[idx]
                log_probs[step][AgentID]=log_prob[idx]
            env.set_actions(behaviorName, a.cpu().detach().numpy() )   
            env.step()
            DecisionSteps, TerminalSteps = env.get_steps(behaviorName)
            #if next step misses some agents, then do not update step
            lst = list(DecisionSteps.agent_id) + list(TerminalSteps.agent_id)
            if(len(list(set(lst))) != NoAgents):
                continue #skip this step, run next while loop without increase step index
            else:
                #collect reward of this action from next decision and terminal steps
                if(len(TerminalSteps) >0):
                    #if next step has terminal agents, then collect terminal agents first
                    r = TerminalSteps.reward
                    r = torch.FloatTensor(r).unsqueeze(1)
                    for idx in range(len(TerminalSteps)):
                        AgentID = TerminalSteps.agent_id[idx]
                        rewards[step][AgentID]=r[idx]
                        masks[step][AgentID]= 0
                #collect from decision steps
                r = DecisionSteps.reward 
                r = torch.FloatTensor(r).unsqueeze(1)
                for idx in range(len(DecisionSteps)):
                    AgentID = DecisionSteps.agent_id[idx] 
                    if(rewards[step][AgentID] == None): #this agent is not in terminal step
                        rewards[step][AgentID]= r[idx]
                        masks[step][AgentID]= 1
                step = step + 1 #increase step and run next while

step  17 has no decision agents !
Decisin agents []
Terminal agents [1]
Reset the training environment!


In [ ]:
env.close()

In [ ]:
print(len(log_probs), log_probs[0].shape)
print(len(values), values[0].shape)
print(len(rewards), rewards[0].shape)
print(len(masks), masks[0].shape)
print(len(states), states[0].shape)
print(len(actions), actions[0].shape)

20 torch.Size([3, 2])
20 torch.Size([3, 1])
20 torch.Size([3, 1])
20 torch.Size([3, 1])
20 torch.Size([3, 8])
20 torch.Size([3, 2])


### Calculate GAE of this training loop

In [ ]:
for step in reversed(range(NoSteps)):
    print(step, end=", ")

19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 

In [ ]:
gae = 0
gamma=0.99
tau=0.95

In [ ]:
# we need to know the value of step 21
next_v = torch.FloatTensor([[0.2]]*NoAgents) 
print(next_v)

tensor([[0.2000],
        [0.2000],
        [0.2000]])


In [ ]:
values1 = values + [next_v]
print(len(values1))

21


In [ ]:
# try step = 19
returns = []
step = 19
print(rewards[step], values1[step + 1] , masks[step] ,values[step])
delta = rewards[step] + gamma * values1[step + 1] * masks[step] - values1[step]
print("delta =", delta)
gae = delta + gamma * tau * masks[step] * gae
print("gae = ", gae)
returns.insert(0, gae + values1[step])
print("returns =", returns)

tensor([[0.2000],
        [0.2000],
        [0.2000]]) tensor([[0.2000],
        [0.2000],
        [0.2000]]) tensor([[0.2000],
        [0.2000],
        [0.2000]]) tensor([[0.2000],
        [0.2000],
        [0.2000]])
delta = tensor([[0.0396],
        [0.0396],
        [0.0396]])
gae =  tensor([[0.0470],
        [0.0470],
        [0.0470]])
returns = [tensor([[0.2470],
        [0.2470],
        [0.2470]])]


In [ ]:
returns = []
for step in reversed(range(NoSteps)):
    delta = rewards[step] + gamma * values1[step + 1] * masks[step] - values1[step]
    gae = delta + gamma * tau * masks[step] * gae
    returns.insert(0, gae + values1[step])

In [ ]:
print(len(returns), returns[0].shape)

20 torch.Size([3, 1])
